In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.metrics import mean_absolute_error 

In [6]:
# Load the data.

#train = pd.read_csv("train_2017.csv")
#test = pd.read_csv("test_2017.csv")

In [10]:
# Load the data.

prop_17 = pd.read_csv('properties_2017.csv')

/home/tcbic/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
sub_prop_17 = prop_17[['bedroomcnt',
                           'bathroomcnt', 'taxvaluedollarcnt', 'calculatedfinishedsquarefeet',
                           'yearbuilt']]

In [12]:
# Bedrooms

sub_prop_17 = sub_prop_17[(sub_prop_17['bedroomcnt'] >= 1) & (sub_prop_17['bedroomcnt'] <= 8)]

In [13]:
# Bathrooms

sub_prop_17 = sub_prop_17[(sub_prop_17['bathroomcnt'] >= 1) & (sub_prop_17['bathroomcnt'] <= 6)]

In [14]:
# Square feet

sub_prop_17 = sub_prop_17[(sub_prop_17['calculatedfinishedsquarefeet'] >= 400) & (sub_prop_17['calculatedfinishedsquarefeet'] <= 10000)]

In [15]:
# Year built

sub_prop_17 = sub_prop_17[(sub_prop_17['yearbuilt'] >= 1900)]

In [16]:
# Tax value

sub_prop_17 = sub_prop_17[(sub_prop_17['taxvaluedollarcnt'] >= 20000) & (sub_prop_17['taxvaluedollarcnt'] <= 10000000)]

In [17]:
# Check that there are no missing values.

sub_prop_17.isnull().sum()

bedroomcnt                      0
bathroomcnt                     0
taxvaluedollarcnt               0
calculatedfinishedsquarefeet    0
yearbuilt                       0
dtype: int64

In [18]:
sub_prop_17.head()

,bedroomcnt,bathroomcnt,taxvaluedollarcnt,calculatedfinishedsquarefeet,yearbuilt
10,4.0,4.0,192544.0,3095.0,2016.0
20,4.0,2.0,296425.0,3633.0,2005.0
28,5.0,4.0,831224.0,2865.0,1972.0
30,2.0,2.0,357568.0,1090.0,2004.0
31,3.0,4.0,847770.0,1620.0,2011.0


In [19]:
# Create a smaller random subset to help with model efficiency.

samp_sub_prop_17 = sub_prop_17.sample(n=10000)

In [22]:
# Check the shape.

samp_sub_prop_17.shape

(10000, 5)

In [ ]:
# Train/Test split...

In [27]:
# Define features and target.

features = samp_sub_prop_17.drop(columns=['taxvaluedollarcnt']).values
target = samp_sub_prop_17['taxvaluedollarcnt'].values

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
# Split X into train and test.

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)

In [30]:
# Check the shape.

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7500, 4), (2500, 4), (7500,), (2500,))

In [36]:
# XGBRegressor Model

import sys
!{sys.executable} -m pip install xgboost

  Using cached https://files.pythonhosted.org/packages/c1/24/5fe7237b2eca13ee0cfb100bec8c23f4e69ce9df852a64b0493d49dae4e0/xgboost-0.90-py2.py3-none-manylinux1_x86_64.whl


In [37]:
import xgboost

In [38]:
from xgboost import XGBRegressor

In [39]:
model2 = XGBRegressor()

In [40]:
model2.fit(X_train, y_train)

[20:35:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [42]:
y_pred2 = model2.predict(X_test)

In [43]:
mean_absolute_error(y_test, y_pred2)

195040.0617625

In [ ]:
# Neural Network Model

In [31]:
nnmodel = Sequential()
# Input => Hidden
nnmodel.add(Dense(32, input_dim=4, activation='relu'))
# Hidden
nnmodel.add(Dense(32, activation='relu'))
#nnmodel.add(Dense(32, activation='relu'))
#nnmodel.add(Dense(32, activation='relu'))
# Output Layer
nnmodel.add(Dense(1, activation='linear'))

# Compile
nnmodel.compile(loss='mean_absolute_error',
                    optimizer='adam',
                    metrics=['mean_absolute_error'])

nnmodel.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                160       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,249
Trainable params: 1,249
Non-trainable params: 0
_________________________________________________________________


In [32]:
nnmodel.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=.3, verbose=1)
scores = nnmodel.evaluate(X_test, y_test)
print(f'{nnmodel.metrics_names[1]}: {scores[1]*100}')

Train on 5250 samples, validate on 2250 samples
Epoch 1/10
5250/5250 [==============================] - 1s 168us/sample - loss: 408581.3096 - mean_absolute_error: 408581.3125 - val_loss: 388601.7759 - val_mean_absolute_error: 388601.6875
Epoch 2/10
5250/5250 [==============================] - 1s 137us/sample - loss: 296436.7915 - mean_absolute_error: 296436.7500 - val_loss: 233608.8138 - val_mean_absolute_error: 233608.8438
Epoch 3/10
5250/5250 [==============================] - 1s 136us/sample - loss: 212821.0311 - mean_absolute_error: 212820.9688 - val_loss: 220870.0031 - val_mean_absolute_error: 220870.0312
Epoch 4/10
5250/5250 [==============================] - 1s 120us/sample - loss: 210403.7907 - mean_absolute_error: 210403.6875 - val_loss: 219479.5275 - val_mean_absolute_error: 219479.5469
Epoch 5/10
5250/5250 [==============================] - 1s 174us/sample - loss: 209197.5929 - mean_absolute_error: 209197.6406 - val_loss: 218207.5418 - val_mean_absolute_error: 218207.5000
Ep

In [ ]:
# Function to create model.

def create_model():
    # create model
    nnmodel = Sequential()
    # Input => Hidden
    nnmodel.add(Dense(32, input_dim=4, activation='relu'))
    # Hidden
    nnmodel.add(Dense(32, activation='relu'))
    nnmodel.add(Dense(32, activation='relu'))
    nnmodel.add(Dense(32, activation='relu'))
    # Output Layer
    nnmodel.add(Dense(1, activation='linear'))

    # Compile
    nnmodel.compile(loss='mean_absolute_error',
                    optimizer='adam',
                    metrics=['mean_absolute_error'])
    return model
  
model = KerasRegressor(build_fn=create_model, verbose=1)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [10,20,40,60,80]}

# Create Grid Search...
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=5)
grid_result = grid.fit(X_train, y_train)

# Report Results...
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

In [34]:
def baseline_model(optimizer='adam'):
    # create model
    model = Sequential()
    model.add(Dense(32, activation='relu', 
                    kernel_regularizer = 'l2', 
                    kernel_initializer = 'normal', 
                    input_shape=(4,)))
   # model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', 
                    kernel_regularizer = 'l2',
                    kernel_initializer = 'normal'))
    #model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear',
                    kernel_regularizer = 'l2', 
                    kernel_initializer='normal'))
    model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

def gridSearch_neural_network(X_train, y_train):
    
    print("Train Data:", X_train.shape)
    print("Train label:", y_train.shape)
    # evaluate model with standardized dataset
    estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=10, batch_size=5, verbose=1)
    
    # grid search epochs, batch size and optimizer
    optimizers = ['rmsprop', 'adam']
    #dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    init = ['glorot_uniform', 'normal', 'uniform']
    epochs = [50, 100, 150]
    batches = [5, 10, 20]
    weight_constraint = [1, 2, 3, 4, 5]
    param_grid = dict(optimizer=optimizers, 
                      #dropout_rate=dropout_rate, 
                      epochs=epochs, 
                      batch_size=batches, 
                      #weight_constraint=weight_constraint, 
                      #init=init
                     )
    
    grid = GridSearchCV(estimator=estimator, param_grid=param_grid)
    grid_result = grid.fit(X_train, y_train)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [35]:
gridSearch_neural_network(X_train, y_train)

Train Data: (7500, 4)
Train label: (7500,)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/50
5000/5000 [==============================] - 5s 956us/sample - loss: 307804.5098 - mean_absolute_error: 307803.6250
Epoch 2/50
5000/5000 [==============================] - 4s 822us/sample - loss: 229986.6588 - mean_absolute_error: 229984.8906
Epoch 3/50
5000/5000 [==============================] - 4s 825us/sample - loss: 228103.8725 - mean_absolute_error: 228102.1250
Epoch 4/50
5000/5000 [==============================] - 4s 837us/sample - loss: 226945.7213 - mean_absolute_error: 226944.0000
Epoch 5/50
5000/5000 [==============================] - 6s 1ms/sample - loss: 222459.9048 - mean_absolute_error: 222458.0938
Epoch 6/50
5000/5000 [==============================] - 3s 691us/sample - loss: 223546.4188 - mean_absolute_error: 223544.5781
Epoch 7/50
5000/5000 [==============================] - 4s 888us/sample - loss:

KeyboardInterrupt: 

In [ ]:
# Pickle model.

In [ ]:
import pickle

In [ ]:
pickle.dump(nnmodel, open('zillow_nn_model.pkl','wb'))